# Build a Chatbot

* https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="こんにちは！私はボブです。")])

In [ ]:
model.invoke([("human", "こんにちは！私はアリスです。")])

In [ ]:
model.invoke([HumanMessage(content="私の名前は何？")])

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="こんにちは！私はボブです。"),
        AIMessage(content="こんにちは、ボブさん！お会いできて嬉しいです。今日はどんなことを話しましょうか？"),
        HumanMessage(content="私の名前は何？"),
    ]
)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    print("***", session_id)
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="こんにちは！私はボブです。")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="私の名前は？")],
    config=config,
)

response.content

In [ ]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="私の名前は？")],
    config=config,
)

response.content

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="私の名前は？")],
    config=config,
)

response.content

In [ ]:
store["abc2"]

In [ ]:
store["abc2"].messages

In [ ]:
store["abc3"].messages

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは親切なアシスタントです。あなたの全力をもってすべての質問に回答してください。"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [ ]:
prompt.invoke({"messages": [HumanMessage(content="こんにちは！私はボブです。")]})

In [ ]:
response = chain.invoke({"messages": [HumanMessage(content="こんにちは！私はボブです。")]})
response.content

In [ ]:
# modelをchainに置き換えられる
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="こんにちは！私はボブです。")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="私の名前は？")],
    config=config,
)

response.content

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは親切なアシスタントです。あなたの全力をもって {language} ですべての質問に回答してください。"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="こんにちは。私の名前はボブです。")],
     "language": "英語"}
)

response.content

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",)

In [ ]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="こんにちは！私はボブです。")],
     "language": "スペイン語"},
     config=config
)

response.content

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="私の名前は？")],
     "language": "スペイン語"},
     config=config
)

response.content

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)

response.content

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's math problem did i ask?")],
        "language": "English"
    }
)

response.content

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    },
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": messages + [HumanMessage(content="Hi! I'm Todd. Tell me a joke.")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")